# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1ogme9BJeHb2IZ6UREavUorF--nnxoWCYAAi8AZ4Q5jQ/edit?usp=sharing) and [grading rubric](https://docs.google.com/document/d/1XI9Yq_e-U-D3iH4jTPAtNteeP2Q9mtJ9NKbePWKeN_g/edit?usp=sharing)

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [1]:
!pip install geoalchemy2
!pip install geopandas

In [2]:
# All import statements needed for the project, for example:
import json
import pathlib
import urllib.parse
import os
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
from shapely.geometry import Point
from sqlalchemy.orm import declarative_base

In [4]:
# Where data files will be read from/written to - this should already exist
# DATA_DIR = pathlib.Path("/Users/catherinewang/Library/CloudStorage/GoogleDrive-xw2925@columbia.edu/My Drive/Final Project 4501/data")
DATA_DIR = pathlib.Path("/Users/luyanni/Library/CloudStorage/GoogleDrive-luyanni2001@gmail.com/My Drive/Final Project 4501/data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "1DrbcO2jXtETwL4T7Hm2ER2Lq"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "pi5s-9p35.geojson"

In [8]:
# DB_NAME = "4501_Data_Analysis"
# DB_USER = "user1"
# DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
# DB_SCHEMA_FILE = "schema.sql"
# # directory where DB queries for Part 3 will be saved
# QUERY_DIR = pathlib.Path("queries")

In [9]:
# # Make sure the QUERY_DIRECTORY & DATA_DIR exists
# if not DATA_DIR.exists():
#     DATA_DIR.mkdir()
# if not QUERY_DIR.exists():
#     QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [4]:
def download_nyc_geojson_data(url, force=False):
    # Parse the given URL
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")

    # Construct the filename based on the URL path
    filename = DATA_DIR / url_path

    # Check if the directory exists, and create it if it does not
    if not filename.parent.exists():
        os.makedirs(filename.parent, exist_ok=True)

    # Download the data if force is True or if the file does not exist
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        try:
            # Request the data from the URL
            response = requests.get(url)
            response.raise_for_status()  # Raises a HTTPError if the response was an error

            # Save the data to the file
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"Done downloading {url}.")
        except Exception as e:
            print(f"Failed to download {url}. Error: {e}")
            return None
    else:
        print(f"File already exists at {filename}.")

    return filename

In [44]:
def load_and_clean_zipcodes(ZIPCODE_DATA_FILE):
    # Load the shapefile
    gdf = gpd.read_file(ZIPCODE_DATA_FILE)

    # Drop rows with missing values
    gdf = gdf.dropna()

    # List of columns to keep
    columns_to_keep = ['ZIPCODE', 'geometry', 'SHAPE_AREA']
    gdf = gdf[columns_to_keep]

    # Remove duplicate ZIP codes
    gdf = gdf.drop_duplicates(subset='ZIPCODE')

    # Filter out ZIP codes that are not in NYC
    valid_nyc_zip_codes = (
        [f"{z:05d}" for z in range(10001, 10293)] +  # Manhattan
        [f"{z:05d}" for z in range(10451, 10476)] +  # Bronx
        [f"{z:05d}" for z in range(11201, 11257)] +  # Brooklyn
        [f"{z:05d}" for z in range(11001, 11698)] +  # Queens
        [f"{z:05d}" for z in range(10301, 10315)]    # Staten Island
    )
    gdf = gdf[gdf['ZIPCODE'].isin(valid_nyc_zip_codes)]

    # Cast columns to appropriate types
    gdf['ZIPCODE'] = gdf['ZIPCODE'].astype(str)

    # Save the cleaned GeoDataFrame back to a new shapefile
    cleaned_file_path = ZIPCODE_DATA_FILE.parent / "nyc_zipcodes_cleaned.shp"
    gdf.to_file(cleaned_file_path)

    return gdf


In [6]:
def download_and_clean_311_data():
    limit = 5000
    offset = 0
    all_gdf = gpd.GeoDataFrame()

    while True:
        query = f"?$where=created_date > '2023-10-01T00:00:00' AND created_date < '2024-03-01T00:00:00' AND latitude IS NOT NULL&$limit={limit}&$offset={offset}&$$app_token={NYC_DATA_APP_TOKEN}"
        full_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}{query}"

        filename = download_nyc_geojson_data(full_url, True)
        if filename is None:
            print(f"Failed to download data for offset {offset}.")
            break

        gdf = gpd.read_file(filename)

        if gdf.empty:
            print("No more data to process.")
            break

        gdf.columns = [col.lower().replace(' ', '_') for col in gdf.columns]

        necessary_columns = ['unique_key', 'created_date', 'incident_zip', 'complaint_type','city', 'geometry']

        gdf = gdf[necessary_columns].dropna(subset=['incident_zip', 'city'])

        gdf['created_date'] = pd.to_datetime(gdf['created_date'])

        all_gdf = pd.concat([all_gdf, gdf], ignore_index=True)

        offset += limit

    if not all_gdf.empty:
        all_gdf.set_crs(epsg=4326, inplace=True)
        geojson_path = DATA_DIR / 'nyc_311_data_cleaned_part6.geojson'
        all_gdf.to_file(geojson_path, driver='GeoJSON')
        print(f"Data saved to {geojson_path}")

    return all_gdf

In [8]:
clean_311_nyc_data = download_and_clean_311_data()

Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date > '2023-10-01T00:00:00' AND created_date < '2024-03-01T00:00:00' AND latitude IS NOT NULL&$limit=5000&$offset=0&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date > '2023-10-01T00:00:00' AND created_date < '2024-03-01T00:00:00' AND latitude IS NOT NULL&$limit=5000&$offset=5000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date > '2023-10-01T00:00:00' AND created_date < '2024-03-01T00:00:00' AND latitude IS NOT NULL&$limit=5000&$offset=10000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/erm2-nwe9.geojson?$where=created_date > '2023-10-01T00:00:00' AND created_date < '2024-03-01T00:00:00' AND latitude IS NOT NULL&$limit=5000&$offset=15000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done down

In [17]:
def download_and_clean_tree_data():
    limit = 5000
    offset = 0
    all_trees_gdf = gpd.GeoDataFrame(columns=['tree_id', 'zipcode', 'geometry', 'status', 'health', 'spc_latin', 'zip_city'])  # Initialize with columns
    all_trees_gdf.set_crs(epsg=4326, inplace=True)  # Set CRS to WGS 84 from the start

    while True:
        query = f"?$where=latitude IS NOT NULL&$limit={limit}&$offset={offset}&$$app_token={NYC_DATA_APP_TOKEN}"
        full_url = f"https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson{query}"
        filename = download_nyc_geojson_data(full_url, True)  # Assume this function returns the filepath correctly

        if not filename:
            break  # Exit the loop if no filename is returned

        gdf_batch = gpd.read_file(filename)

        if gdf_batch.empty:
            print("No more data to process.")
            break

        gdf_batch.columns = [col.lower().replace(' ', '_') for col in gdf_batch.columns]

        # Filter for necessary columns, including latitude and longitude for geometry creation
        columns_to_keep = ['tree_id', 'zipcode', 'latitude', 'longitude', 'status', 'health', 'spc_latin', 'zip_city']
        gdf_batch = gdf_batch[columns_to_keep].dropna(subset=['zipcode', 'zip_city'])

        # Convert latitude and longitude to float
        gdf_batch['latitude'] = gdf_batch['latitude'].astype(float)
        gdf_batch['longitude'] = gdf_batch['longitude'].astype(float)

        # Create geometry column from latitude and longitude
        gdf_batch['geometry'] = [Point(xy) for xy in zip(gdf_batch.longitude, gdf_batch.latitude)]
        # Drop the latitude and longitude columns as they are no longer needed
        gdf_batch.drop(columns=['latitude', 'longitude'], inplace=True)
        # Create a GeoDataFrame with the geometry column
        gdf_batch = gpd.GeoDataFrame(gdf_batch, geometry='geometry')
        gdf_batch.set_crs(epsg=4326, inplace=True)  # Ensure the GeoDataFrame is in WGS 84 CRS

        # Concatenate with the main GeoDataFrame
        all_trees_gdf = pd.concat([all_trees_gdf, gdf_batch], ignore_index=True)

        offset += limit  # Increment the offset for the next batch

    if not all_trees_gdf.empty:
        # Save the cleaned GeoDataFrame to a GeoJSON file, now that geometry is used
        cleaned_filepath = DATA_DIR / 'nyc_trees_data_cleaned.geojson'
        all_trees_gdf.to_file(cleaned_filepath, driver='GeoJSON')
        print(f"Cleaned tree data saved to {cleaned_filepath}")

    return all_trees_gdf


In [18]:
clean_tree_nyc_data = download_and_clean_tree_data()

Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=0&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=1000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=2000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=3000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=4000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done downloading https://data.cityofnewyork.us/resource/uvpi-gqnh.geojson?$where=latitude IS NOT NULL&$limit=1000&$offset=5000&$$app_token=1DrbcO2jXtETwL4T7Hm2ER2Lq.
Done do

In [18]:
def load_combined_311_data():
    geo_df_311 = gpd.read_file(DATA_DIR / "nyc_311_data_cleaned_combined.geojson")
    return geo_df_311  

In [29]:
def load_tree_data():
    geo_df_tree = gpd.read_file(DATA_DIR / "nyc_trees_data_cleaned.geojson")
    return geo_df_tree

In [30]:
geodf_tree_data = load_tree_data()

In [32]:
city_tree = geodf_tree_data['zip_city'].dropna().unique()

In [33]:
city_tree_lower = [item.lower() for item in city_tree]

In [19]:
geodf_311_data = load_combined_311_data()

In [24]:
geodf_311_data

,unique_key,created_date,incident_zip,complaint_type,city,geometry
0,54334222,2022-05-31 23:59:56,10454,Illegal Parking,BRONX,POINT (-73.91758 40.81191)
1,54404953,2022-05-31 23:59:46,10023,Elevator,NEW YORK,POINT (-73.98334 40.78271)
2,54330771,2022-05-31 23:59:44,11219,Illegal Parking,BROOKLYN,POINT (-74.00173 40.62423)
3,54330124,2022-05-31 23:59:08,11420,Illegal Parking,SOUTH OZONE PARK,POINT (-73.82238 40.67466)
4,54339731,2022-05-31 23:58:39,10466,Noise - Residential,BRONX,POINT (-73.85922 40.89238)
...,...,...,...,...,...,...
5807512,58987186,2023-10-01 00:00:37,11233,FLOORING/STAIRS,BROOKLYN,POINT (-73.91816 40.67436)
5807513,58981257,2023-10-01 00:00:35,10452,APPLIANCE,BRONX,POINT (-73.93107 40.83168)
5807514,58972578,2023-10-01 00:00:22,11220,Noise - Vehicle,BROOKLYN,POINT (-74.01426 40.63646)
5807515,58973513,2023-10-01 00:00:04,11419,Noise - Residential,SOUTH RICHMOND HILL,POINT (-73.82167 40.69077)


In [20]:
city_311 = geodf_311_data['city'].dropna().unique()

In [23]:
city_311_lower = [item.lower() for item in city_311]
city_311_lower

['bronx',
 'new york',
 'brooklyn',
 'south ozone park',
 'staten island',
 'glen oaks',
 'corona',
 'jackson heights',
 'east elmhurst',
 'jamaica',
 'far rockaway',
 'astoria',
 'flushing',
 'south richmond hill',
 'fresh meadows',
 'elmhurst',
 'whitestone',
 'sunnyside',
 'cambria heights',
 'ridgewood',
 'springfield gardens',
 'oakland gardens',
 'queens village',
 'woodhaven',
 'queens',
 'manhattan',
 'long island city',
 'ozone park',
 'maspeth',
 'forest hills',
 'saint albans',
 'rego park',
 'middle village',
 'richmond hill',
 'rosedale',
 'howard beach',
 'arverne',
 'woodside',
 'college point',
 'hollis',
 'bellerose',
 'bayside',
 'little neck',
 'kew gardens',
 'floral park',
 'new hyde park',
 'breezy point',
 'long island city',
 'laguardia airport',
 'queens',
 'queens village',
 'ozone park',
 'new york city',
 'brooklyn',
 'new york',
 'bronx',
 'forest hills',
 'pelham',
 'jamaica',
 'jamaica plain',
 'boca raton',
 'mason',
 'mt. vernon',
 'woodside',
 'new yor

In [34]:
combined_set = set(city_311_lower + city_tree_lower)
combined_list = list(combined_set)
combined_list

['middle village',
 'queens village',
 'howard beach',
 'laguardia airport',
 'boca raton',
 'sunnyside',
 'fresh meadows',
 'astoria',
 'richmond hill',
 'rockaway park',
 'flushing',
 'ridgewood',
 'jamaica plain',
 'bayside',
 'jamiaca queens',
 'whitestone',
 'manhattan',
 'hollis',
 'rego park',
 'mt. vernon',
 'bronx',
 'long island city',
 'east elmhurst',
 'saint albans',
 'oakland gardens',
 'woodside',
 'college point',
 'floral park',
 'breezy point',
 'pelham',
 'queens',
 'staten island',
 'little neck',
 'kew gardens',
 'inwood',
 'jamaica',
 'south ozone park',
 'rosedale',
 'arverne',
 'bellerose',
 'mason',
 'new york',
 'brooklyn',
 'cambria heights',
 'central park',
 'ozone park',
 'new hyde park',
 'new york city',
 'elmhurst',
 'jackson heights',
 'springfield gardens',
 'forest hills',
 'woodhaven',
 'south richmond hill',
 'corona',
 'glen oaks',
 'far rockaway',
 'maspeth']

In [11]:
df_tree = gpd.read_file(DATA_DIR / "nyc_trees_data_cleaned.geojson")

In [13]:
# def load_and_clean_zillow_data(NYC_Cities):
#     # Load the data into a DataFrame
#     zillow_rent_df = pd.read_csv(ZILLOW_DATA_FILE)
    
#     # Assuming the date columns are in a format like 'YYYY-MM-DD'
#     # First, identify all columns that are not date columns but need to be kept (e.g., 'RegionName', 'City')
#     non_date_columns = ['RegionName', 'City', 'SizeRank']

#     # Dynamically generate a list of date columns to keep, based on the specified date range
#     # Convert all column names to datetime where possible; this will yield NaT for non-date columns
#     date_columns = pd.to_datetime(zillow_rent_df.columns, errors='coerce', format='%Y-%m-%d')

#     # Identify columns within the specified date range
#     start_date = pd.to_datetime('2022-02-01')
#     end_date = pd.to_datetime('2024-01-31')
#     date_columns_to_keep = zillow_rent_df.columns[(date_columns >= start_date) & (date_columns <= end_date)]

#     # Combine the non-date columns to keep with the date columns identified in the range
#     columns_to_keep = non_date_columns + date_columns_to_keep.tolist()

#     # Filter the DataFrame to include only the identified columns
#     zillow_rent_df = zillow_rent_df[columns_to_keep]

#     # Filter rows where 'City' is in the NYC_Cities list
#     zillow_rent_df = zillow_rent_df[zillow_rent_df['City'].str.lower().isin([city.lower() for city in NYC_Cities])]

#     return zillow_rent_df.dropna()

In [36]:
def load_and_clean_zillow_data(NYC_Cities):
    # Load the data into a DataFrame
    zillow_rent_df = pd.read_csv(ZILLOW_DATA_FILE)
    
    # Assuming the date columns are in a format like 'YYYY-MM-DD'
    non_date_columns = ['RegionName', 'City', 'SizeRank']
    date_columns = pd.to_datetime(zillow_rent_df.columns, errors='coerce', format='%Y-%m-%d')

    # Define the date range
    start_date = pd.to_datetime('2022-02-01')
    end_date = pd.to_datetime('2024-01-31')
    date_columns_to_keep = zillow_rent_df.columns[(date_columns >= start_date) & (date_columns <= end_date)]

    # Combine the columns to keep
    columns_to_keep = non_date_columns + date_columns_to_keep.tolist()
    zillow_rent_df = zillow_rent_df[columns_to_keep]

    # Filter by NYC_Cities, converting all city names in the DataFrame and list to lower case for case insensitive comparison
    zillow_rent_df = zillow_rent_df[zillow_rent_df['City'].str.lower().isin([city.lower() for city in NYC_Cities])]

    # Drop rows with missing values
    cleaned_df = zillow_rent_df.dropna()

    # Save the cleaned DataFrame to a file
    cleaned_filepath = DATA_DIR / 'cleaned_zillow_data.csv'
    cleaned_df.to_csv(cleaned_filepath, index=False)
    print(f"Cleaned Zillow data saved to {cleaned_filepath}")

    return cleaned_df



In [37]:
df_zillow_data = load_and_clean_zillow_data(combined_list)

Cleaned Zillow data saved to /Users/luyanni/Library/CloudStorage/GoogleDrive-luyanni2001@gmail.com/My Drive/Final Project 4501/data/cleaned_zillow_data.csv


In [48]:
def load_zillow_data():
    # Read the CSV data file using pandas
    df = pd.read_csv(DATA_DIR / "cleaned_zillow_data.csv")
    return df

In [16]:
zillow = load_and_clean_zillow_data(city_311_lower)
zillow['City'].unique()

array(['New York', 'Corona', 'Mason', 'Manhattan', 'Elmhurst',
       'Boca Raton', 'Richmond Hill'], dtype=object)

In [45]:
geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

In [49]:
def load_all_data():
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = load_combined_311_data()
    geodf_tree_data = load_tree_data()
    df_zillow_data = load_zillow_data()
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        df_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, df_zillow_data = load_all_data()

In [46]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 247 entries, 0 to 262
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZIPCODE     247 non-null    object  
 1   geometry    247 non-null    geometry
 2   SHAPE_AREA  247 non-null    float64 
dtypes: float64(1), geometry(1), object(1)
memory usage: 7.7+ KB


In [47]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,ZIPCODE,geometry,SHAPE_AREA
0,11436,"POLYGON ((1038098.252 188138.380, 1038141.936 ...",0.0
1,11213,"POLYGON ((1001613.713 186926.440, 1002314.243 ...",0.0
2,11212,"POLYGON ((1011174.276 183696.338, 1011373.584 ...",0.0
3,11225,"POLYGON ((995908.365 183617.613, 996522.848 18...",0.0
4,11218,"POLYGON ((991997.113 176307.496, 992042.798 17...",0.0


In [25]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5807517 entries, 0 to 5807516
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   unique_key      object        
 1   created_date    datetime64[ns]
 2   incident_zip    object        
 3   complaint_type  object        
 4   city            object        
 5   geometry        geometry      
dtypes: datetime64[ns](1), geometry(1), object(4)
memory usage: 265.8+ MB


In [26]:
geodf_311_data.head()

,unique_key,created_date,incident_zip,complaint_type,city,geometry
0,54334222,2022-05-31 23:59:56,10454,Illegal Parking,BRONX,POINT (-73.91758 40.81191)
1,54404953,2022-05-31 23:59:46,10023,Elevator,NEW YORK,POINT (-73.98334 40.78271)
2,54330771,2022-05-31 23:59:44,11219,Illegal Parking,BROOKLYN,POINT (-74.00173 40.62423)
3,54330124,2022-05-31 23:59:08,11420,Illegal Parking,SOUTH OZONE PARK,POINT (-73.82238 40.67466)
4,54339731,2022-05-31 23:58:39,10466,Noise - Residential,BRONX,POINT (-73.85922 40.89238)


In [38]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   tree_id    683788 non-null  object  
 1   zipcode    683788 non-null  object  
 2   status     683788 non-null  object  
 3   health     652172 non-null  object  
 4   spc_latin  652169 non-null  object  
 5   zip_city   683788 non-null  object  
 6   geometry   683788 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 36.5+ MB


In [39]:
geodf_tree_data.head()

,tree_id,zipcode,status,health,spc_latin,zip_city,geometry
0,180683,11375,Alive,Fair,Acer rubrum,Forest Hills,POINT (-73.84422 40.72309)
1,200540,11357,Alive,Fair,Quercus palustris,Whitestone,POINT (-73.81868 40.79411)
2,204026,11211,Alive,Good,Gleditsia triacanthos var. inermis,Brooklyn,POINT (-73.93661 40.71758)
3,204337,11211,Alive,Good,Gleditsia triacanthos var. inermis,Brooklyn,POINT (-73.93446 40.71354)
4,189565,11215,Alive,Good,Tilia americana,Brooklyn,POINT (-73.97598 40.66678)


In [40]:
df_zillow_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 5 to 6630
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RegionName  114 non-null    int64  
 1   City        114 non-null    object 
 2   SizeRank    114 non-null    int64  
 3   2022-02-28  114 non-null    float64
 4   2022-03-31  114 non-null    float64
 5   2022-04-30  114 non-null    float64
 6   2022-05-31  114 non-null    float64
 7   2022-06-30  114 non-null    float64
 8   2022-07-31  114 non-null    float64
 9   2022-08-31  114 non-null    float64
 10  2022-09-30  114 non-null    float64
 11  2022-10-31  114 non-null    float64
 12  2022-11-30  114 non-null    float64
 13  2022-12-31  114 non-null    float64
 14  2023-01-31  114 non-null    float64
 15  2023-02-28  114 non-null    float64
 16  2023-03-31  114 non-null    float64
 17  2023-04-30  114 non-null    float64
 18  2023-05-31  114 non-null    float64
 19  2023-06-30  114 non-null    

In [41]:
df_zillow_data.head()

,RegionName,City,SizeRank,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,...,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
5,11385,New York,7,2511.479296,2523.393469,2567.347532,2643.069051,2741.305526,2805.714576,2861.792781,...,2808.032817,2853.002812,2891.976411,2912.450226,2938.806070,2955.446452,2944.372938,2918.206892,2888.346480,2880.094236
14,10467,New York,17,1607.726640,1647.610455,1671.162249,1664.858068,1677.572862,1739.992403,1756.424497,...,1695.177852,1748.353876,1825.836631,1883.032947,1899.143080,1958.261105,1987.880524,2066.328960,2011.173298,2036.252778
15,11373,New York,18,2274.638306,2340.801631,2381.756319,2407.443451,2444.372215,2513.450528,2496.212481,...,2595.856831,2570.915900,2608.660190,2596.701012,2660.168354,2642.649107,2670.325604,2668.151494,2672.319180,2689.050000
16,11226,New York,19,2415.357857,2451.406046,2492.544828,2528.553196,2585.077670,2611.357874,2620.821184,...,2613.659650,2633.525698,2662.290709,2679.586231,2693.602946,2672.819878,2644.929685,2644.410748,2652.155298,2680.780024
17,11207,New York,20,2503.056202,2499.803803,2535.860175,2643.947722,2760.415916,2859.214463,2883.894035,...,2839.791966,2838.526876,2916.117310,2976.541426,3042.519873,3129.123626,3101.798032,3052.525088,2931.655657,2931.278704


## Part 2: Storing Data

In [ ]:
!pip install psycopg2
!pip install psycopg2-binary

In [ ]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def setup_new_postgis_database(username, db_name, user_password):
    superuser = "postgres"
    superuser_password = "superuser_password"

    try:
        # Connect to PostgreSQL DBMS as superuser
        conn = psycopg2.connect(user=superuser, password=superuser_password)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

        # Create a cursor object using the connection
        cur = conn.cursor()

        # Create database
        cur.execute(sql.SQL("CREATE DATABASE {}").format(
            sql.Identifier(db_name)))
        print(f"Database {db_name} created successfully.")

        # Create user
        cur.execute(sql.SQL("CREATE USER {} WITH PASSWORD {}").format(
            sql.Identifier(username),
            sql.Literal(user_password)))
        print(f"User {username} created successfully.")

        # Close the cursor and initial connection
        cur.close()
        conn.close()

        # Connect to the newly created database to enable PostGIS
        conn = psycopg2.connect(dbname=db_name, user=superuser, password=superuser_password)
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        cur = conn.cursor()

        # Enable PostGIS (assumes PostGIS is installed)
        cur.execute(sql.SQL("CREATE EXTENSION IF NOT EXISTS postgis"))
        print(f"PostGIS extension enabled on database {db_name}.")

        # Clean up
        cur.close()
        conn.close()

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
username = "user1"
db_name = "4501_Data_Analysis"
user_password = "Aa1223456"
setup_new_postgis_database(username, db_name, user_password)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# If using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables.
# You may be creating more tables depending on how you're setting up your constraints/relationships
# or if you're completing the extra credit.
ZIPCODE_SCHEMA = """
TODO
"""

ZIPCODE_G

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)